<a href="https://colab.research.google.com/github/srpantano/deep-learningaz/blob/master/breast_cancer_simples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> ## Importa as bibliotecas

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import keras
from keras.models import Sequential
from keras.layers import Dense

> ## Importa as base de dados e divide entre treino e teste

In [0]:
X = pd.read_csv('https://raw.githubusercontent.com/srpantano/Learnpath/master/entradas-breast.csv')
y = pd.read_csv('https://raw.githubusercontent.com/srpantano/Learnpath/master/saidas-breast.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

> ## Cria a rede neural

In [0]:
classifier = Sequential()
'''Número de classes é 30. A saída é binário (é ou não malígno) = 
units é o número de neurônios na camada
(30 classes + 1 saída) / 2 = 15.5, arredonda para 16
relu é a função de ativação mais indicada para camadas ocultas de Deep Learning
input_dim = Quantidade de classes de entrada
Por padrão ele possui um parâmetro use_bias=True, para utilizar Bias'''
classifier.add(Dense(units = 16, activation='relu', 
                     kernel_initializer='random_uniform', 
                     input_dim = 30))
classifier.add(Dense(units = 16, activation='relu', 
                     kernel_initializer='random_uniform'))
#Saída binária, sigmoid para ser entre 0 e 1
classifier.add(Dense(units = 1, activation = 'sigmoid'))

'''clipvalue e clipenorm são parametros globais dos otimizadores. Eles servem como
fixadores para os saltos na bisca do mínimo global'''
'''Learning rate (lr) alto pode fazer com que ele "pule" o mímno. Já o alto é mais preciso, porém lento.
Default lr = 0.001
Decaimento (decay) o quanto ele irá decair a cada iteração.'''
optimizerCustom = keras.optimizers.Adam(lr = 0.001, decay = 0.0001, clipvalue = 0.5)
classifier.compile(optimizer = optimizerCustom, loss = 'binary_crossentropy', 
                   metrics = ['binary_accuracy']) #Utilizando o otimizador customizado

#classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', 
#                   metrics = ['binary_accuracy'])

> ## Faz o treinamento da rede

In [0]:
#batch_size = 10, significa que ele ajusta os pesos a cada 10 registros.
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

> ## Recupera os pesos da rede

Ele cria uma lista com 2 linhas:  
 
> 1.   pesos das conexões
> 2.   Pesos dos Bias

In [0]:
weights0 = classifier.layers[0].get_weights() #pesos da primeira camada oculta
weights1 = classifier.layers[1].get_weights() #pesos da segunda camada oculta
weightsExit = classifier.layers[2].get_weights() #pesos da segunda camada de saída

In [0]:
# print(weights0)
print('Tamanho dos pesos da primeira camada: ' + str(len(weights0)))
print('Quantidade de pesos da primeira camada: {}'.format(len(weights0[0])))
print('Quantidade de Bias da primeira camada: {}'.format(len(weights0[1])))
print('Tamanho dos pesos da segunda camada: ' + str(len(weights1)))
print('Quantidade de pesos da segunda camada: {}'.format(len(weights1[0])))
print('Quantidade de Bias da segunda camada: {}'.format(len(weights1[1])))
print('Tamanho dos pesos da camada de saída: ' + str(len(weightsExit)))
print('Quantidade de pesos da camada de saída: {}'.format(len(weightsExit[0])))
print('Quantidade de Bias da camada de saída: {}'.format(len(weightsExit[1])))

>## Realiza as previsões dos dados de teste, de acordo com o que foi treinado.

In [0]:
yHat = classifier.predict(X_test)
yHat  = (yHat > 0.5) #True se for acima de 0.5, se não False.

>## Exibindo os resultados utilzando o Sckit-learn

In [0]:
precision = accuracy_score(y_test, yHat)
matrix_confusion = confusion_matrix(y_test, yHat)
print(precision)
print(matrix_confusion)

>## Exibindo os resultados utilzando o Keras

In [0]:
result = classifier.evaluate(X_test, y_test)
print(result)

>## Salva o modelo construido

In [0]:
from keras.utils import plot_model
plot_model(classifier, to_file='model.png')